# Imports

In [1]:
from pathlib import Path


import gc
import pandas as pd
import re
import requests
gc.enable()

/Users/asoni9/Desktop/work/flute-core/env/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


- outro/intro/filler/sponsor/interaction/selfpromo
- channel_id filter

#### For Modeling -:

Please refer with sklearn

- kNN
- DBSCAN


#### Feature Engineering

- Create ratios
    - sub-segment / total video length
    - time_delta features like how much time in sponsor / other cateogories
        - for sponsors
        - for all other views

- fastText (text columns) -> convert the title to embeddings (KNN) etc (glove / fastText embeddings etc/)
- Clusters will be for (sponsor, self_promo, outro + intro + interaction + filler = interaction)

In [2]:
BASE_PATH = Path("./tmp")
DATA_PATH = BASE_PATH / "data"

channel_name_to_id = {}
REGEX_CHANNEL_ID = re.compile(r"channel_id=(\w+-?\w+)\"\>")

CHANNELS_TO_FILTER = [
    "AkshatZayn",
    "AssetYogi",
    "CARachanaRanade",
    "shankarnath",
    "PRSundar64",
    "InvestYadnya",
    "ParimalAde",
    "namaskarprasad",
    "pranjalkamra",
    "PushkarRajThakurOfficial",
    "warikoo"
]


# we need channel_ids to filter the underlying dataset, fetching that information from YT
for channel_name in CHANNELS_TO_FILTER:
    try:
        page_source = requests.get(f"https://www.youtube.com/@{channel_name}").text
        channel_id = REGEX_CHANNEL_ID.findall(page_source)
        print(channel_name, channel_id)
        channel_name_to_id[channel_name] = channel_id[0]
    except Exception as error:
        print(f"Failed to find the channel_id for the channel_name {channel_name}")

# We are starting out with 3 clusters
# 1st and 2nd ones are "sponsor" and "selfpromo"
# #rd cluster we are combining rest of the categories mentioned in the below list as one category


CHANNEL_IDS_FILTER = list(
    channel_name_to_id.values()
)

VIDEO_CATEGORY_FILTER = [
    #########
    "outro",
    "intro",
    "filler",
    "interaction",
    
    #########
    "sponsor",
    
    #########,
    "selfpromo"
]


VIDEO_CATEGORY_FILTER_MAP = {
    "outro": 0,
    "intro": 0,
    "filler": 0,
    "interaction": 0,
    "sponsor": 1,
    "selfpromo": 2,
}

AkshatZayn ['UCqW8jxh4tH1Z1sWPbkGWL4g']
AssetYogi ['UCsNxHPbaCWL1tKw2hxGQD6g']
CARachanaRanade ['UCe3qdG0A_gr-sEdat5y2twQ']
shankarnath ['UCtnItzU7q_bA1eoEBjqcVrw']
PRSundar64 ['UCS2NdYUmv_PUyyKeDAo5zYA']
InvestYadnya ['UCPohbSYq4IXhv0yxiy-sT4g']
ParimalAde ['UCUWbS9qoyLKTdOlvgK1GBlA']
namaskarprasad ['UCUMccND2H_CVS0dMZKCPCXA']
pranjalkamra ['UCwAdQUuPT6laN-AQR17fe1g']
PushkarRajThakurOfficial ['UCEAAzv2OBqxsSczKJ2QZyGQ']
warikoo ['UCRzYN32xtBf3Yxsx5BvJWJw']


In [3]:
channel_name_to_id

{'AkshatZayn': 'UCqW8jxh4tH1Z1sWPbkGWL4g',
 'AssetYogi': 'UCsNxHPbaCWL1tKw2hxGQD6g',
 'CARachanaRanade': 'UCe3qdG0A_gr-sEdat5y2twQ',
 'shankarnath': 'UCtnItzU7q_bA1eoEBjqcVrw',
 'PRSundar64': 'UCS2NdYUmv_PUyyKeDAo5zYA',
 'InvestYadnya': 'UCPohbSYq4IXhv0yxiy-sT4g',
 'ParimalAde': 'UCUWbS9qoyLKTdOlvgK1GBlA',
 'namaskarprasad': 'UCUMccND2H_CVS0dMZKCPCXA',
 'pranjalkamra': 'UCwAdQUuPT6laN-AQR17fe1g',
 'PushkarRajThakurOfficial': 'UCEAAzv2OBqxsSczKJ2QZyGQ',
 'warikoo': 'UCRzYN32xtBf3Yxsx5BvJWJw'}

In [4]:
CHANNEL_IDS_FILTER

['UCqW8jxh4tH1Z1sWPbkGWL4g',
 'UCsNxHPbaCWL1tKw2hxGQD6g',
 'UCe3qdG0A_gr-sEdat5y2twQ',
 'UCtnItzU7q_bA1eoEBjqcVrw',
 'UCS2NdYUmv_PUyyKeDAo5zYA',
 'UCPohbSYq4IXhv0yxiy-sT4g',
 'UCUWbS9qoyLKTdOlvgK1GBlA',
 'UCUMccND2H_CVS0dMZKCPCXA',
 'UCwAdQUuPT6laN-AQR17fe1g',
 'UCEAAzv2OBqxsSczKJ2QZyGQ',
 'UCRzYN32xtBf3Yxsx5BvJWJw']

# Reading the dataset

In [5]:
# Below is the dataset schema
# We're currently using only 3 files in the interest of time
# sponsorTimes.csv, titles.csv and videoInfo.csv

In [8]:
# SAMPLE : Simple flag to control how many rows we're reading

SAMPLE = False
N_ROWS = 2**10 if SAMPLE else None

FE_PREFIX = "fe_"


SPONSOR_FILTER_COLUMNS = [
    "actionType",
    "videoID",
    "startTime",
    "endTime",
    "videoDuration",
    "votes",
    "timeSubmitted",
    "views",
    "category",
    "hidden",
    "locked",
    "reputation",
]

FINAL_COLS = []


df_video_info = pd.read_csv(
    DATA_PATH / "videoInfo.csv", 
    nrows = N_ROWS
)

# Filtering the data wrt the channel_ids we're interested in only
df_video_info = df_video_info[df_video_info["channelID"].isin(CHANNEL_IDS_FILTER)]
df_video_info.reset_index(drop=True, inplace=True)
VIDEO_IDS = set(df_video_info["videoID"].unique())


# reading datasets in memory using pandas
df_sponsor_times = pd.read_csv(
    DATA_PATH / "sponsorTimes.csv", 
    nrows = N_ROWS, 
    usecols=SPONSOR_FILTER_COLUMNS
)
df_sponsor_times = df_sponsor_times[df_sponsor_times["videoID"].isin(VIDEO_IDS)]
df_sponsor_times.sort_values(by=["videoID", "startTime"], inplace=True)
df_sponsor_times.reset_index(drop=True, inplace=True)

# Correcting data wrt videoDuration (swapping with maxEndTime available)
df_temp = (
    df_sponsor_times[df_sponsor_times["videoDuration"] <= 0][["videoID", "endTime"]]
    .groupby("videoID")
    .agg(max)
    .reset_index(drop=False)
)
video_id_max_endtime_map = dict(zip(df_temp["videoID"], df_temp["endTime"]))

df_sponsor_times.loc[
    df_sponsor_times.loc[
        (df_sponsor_times["videoID"].isin(video_id_max_endtime_map)) 
        & (df_sponsor_times["videoDuration"] <= 0)
    ].index,  "videoDuration"] = df_sponsor_times["videoID"].map(video_id_max_endtime_map)

df_titles = pd.read_csv(
    DATA_PATH / "titles.csv", 
    nrows = N_ROWS
)

del df_temp, video_id_max_endtime_map

gc.collect()
df_sponsor_times.shape, df_video_info.shape, df_titles.shape

((5475, 12), (2419, 4), (30306, 8))

In [10]:
df_titles.head(1)

,videoID,title,original,userID,service,hashedVideoID,timeSubmitted,UUID
0,U0wTDK0VOeY,History of Antarctica's Flag,0,1ff2cdd11ed952d0c13d678413113860ff279a3d90d31c...,YouTube,93646c719490256e8cb43cfaa41e39534525389b0b28f5...,1680044079895,07f32a99-3e2d-48a3-a247-b2b03f7cd4bc


In [11]:
df_video_info.head(1)

,videoID,channelID,title,published
0,2USGSuPe8SQ,UCsNxHPbaCWL1tKw2hxGQD6g,"Stock Market के Basics, Risks और Returns - Sha...",1.568938e+09


In [12]:
df_sponsor_times.head(5)

,videoID,startTime,endTime,votes,locked,timeSubmitted,views,category,actionType,videoDuration,hidden,reputation
0,--kZomtrtIQ,459.290,465.046,0,0,1666424633336,0,outro,skip,465.046,0,0.000000
1,-2MyBawvlts,26.726,32.713,0,0,1670724050394,1,filler,skip,636.064,0,0.000000
2,-3AfFa0rV6Q,1405.890,1447.159,0,0,1659371242341,0,selfpromo,skip,2477.000,0,1.110345
3,-3AfFa0rV6Q,2088.830,2110.900,0,0,1659371242341,0,sponsor,skip,2477.000,0,1.110345
4,-3AfFa0rV6Q,2366.269,2390.471,0,0,1659371242341,0,interaction,skip,2477.000,0,1.110345


In [13]:
df_sponsor_times[df_sponsor_times["videoID"] == "-3AfFa0rV6Q"]

,videoID,startTime,endTime,votes,locked,timeSubmitted,views,category,actionType,videoDuration,hidden,reputation
2,-3AfFa0rV6Q,1405.890,1447.159,0,0,1659371242341,0,selfpromo,skip,2477.0,0,1.110345
3,-3AfFa0rV6Q,2088.830,2110.900,0,0,1659371242341,0,sponsor,skip,2477.0,0,1.110345
4,-3AfFa0rV6Q,2366.269,2390.471,0,0,1659371242341,0,interaction,skip,2477.0,0,1.110345


In [ ]:
# some filtering
# removing the videos that have full-video label and category we want...

df_sponsor_times = df_sponsor_times[df_sponsor_times["actionType"] != "full"]
df_sponsor_times["locked"] = (df_sponsor_times["locked"] == "1") * 1
df_sponsor_times = df_sponsor_times[df_sponsor_times["category"].isin(VIDEO_CATEGORY_FILTER)]

# Mapping category to numeric values
print(VIDEO_CATEGORY_FILTER_MAP)
df_sponsor_times["category"] = df_sponsor_times["category"].map(VIDEO_CATEGORY_FILTER_MAP)

# Time-Delta-features
df_sponsor_times[f"{FE_PREFIX}segment_duration"] = df_sponsor_times["endTime"] - df_sponsor_times["startTime"]
df_sponsor_times[f"{FE_PREFIX}submission_time"] = df_sponsor_times["timeSubmitted"] / 1e3

df_temp = df_sponsor_times.pivot_table(
        index=["videoID"],
        values=["fe_segment_duration", "views", "votes", "videoDuration"],
        aggfunc={
            "fe_segment_duration": sum,
            "views": sum,
            "votes": sum,
            "videoDuration": max,
        },
        columns=["category"],
        fill_value=0
    )

df_sponsor_times_video_id_level = pd.DataFrame(df_temp.to_records())

column_names = ["videoID"]
for col in df_sponsor_times_video_id_level.columns.values[1:]:
    col_1, category = eval(col)
    column_names.append(col_1 + "_" + str(category))

df_sponsor_times_video_id_level.columns = column_names
df_sponsor_times_video_id_level.drop(
    [
        "videoDuration_0",
        "videoDuration_1",
        "videoDuration_2"
    ],
    axis=1,
    inplace=True
)

video_duration_map = dict(zip(df_sponsor_times["videoID"], df_sponsor_times["videoDuration"]))
df_sponsor_times_video_id_level["videoDuration"] = df_sponsor_times_video_id_level["videoID"].map(video_duration_map)


for category in [0, 1, 2]:
    df_sponsor_times_video_id_level[
        f"{FE_PREFIX}ratio_segment_duration_with_total_video_duration_{category}"
    ] = df_sponsor_times_video_id_level[f"{FE_PREFIX}segment_duration_{category}"] / df_sponsor_times_video_id_level["videoDuration"]
    
    
# more than 35% is sponsor?
df_sponsor_times_video_id_level[f"{FE_PREFIX}more_than_35_perc_sponsorship_content"] = 0
df_sponsor_times_video_id_level["temp_1"] = (
    df_sponsor_times_video_id_level["fe_ratio_segment_duration_with_total_video_duration_0"]
    + df_sponsor_times_video_id_level["fe_ratio_segment_duration_with_total_video_duration_1"]
    + df_sponsor_times_video_id_level["fe_ratio_segment_duration_with_total_video_duration_2"]
)

df_sponsor_times_video_id_level.loc[
    df_sponsor_times_video_id_level["temp_1"] >= 0.344444449,
    f"{FE_PREFIX}more_than_35_perc_sponsorship_content"
] = 1

df_sponsor_times_video_id_level.drop(["temp_1"], axis=1, inplace=True)

In [16]:
df_sponsor_times_video_id_level

,videoID,fe_segment_duration_0,fe_segment_duration_1,fe_segment_duration_2,views_0,views_1,views_2,votes_0,votes_1,votes_2,videoDuration,fe_ratio_segment_duration_with_total_video_duration_0,fe_ratio_segment_duration_with_total_video_duration_1,fe_ratio_segment_duration_with_total_video_duration_2,fe_more_than_35_perc_sponsorship_content
0,--kZomtrtIQ,5.756,0.000,0.000,0,0,0,0,0,0,465.046,0.012377,0.000000,0.000000,0
1,-2MyBawvlts,5.987,0.000,0.000,1,0,0,0,0,0,636.064,0.009413,0.000000,0.000000,0
2,-3AfFa0rV6Q,24.202,22.070,41.269,0,0,0,0,0,0,2477.000,0.009771,0.008910,0.016661,0
3,-3Q-k4WQTDI,7.384,0.000,0.000,0,0,0,0,0,0,242.024,0.030509,0.000000,0.000000,0
4,-6WBAaHqT8g,5.334,0.000,0.000,0,0,0,0,0,0,242.024,0.022039,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358,zwLHJlNMlf4,0.000,24.599,0.000,0,76,0,0,0,0,1092.000,0.000000,0.022527,0.000000,0
2359,zxKURXHy6es,46.073,0.000,0.000,0,0,0,0,0,0,733.073,0.062849,0.000000,0.000000,0
2360,zxdOcHOrAdE,0.000,20.980,0.000,0,0,0,0,0,0,61.060,0.000000,0.343596,0.000000,0
2361,zxi7Rm-lWTg,43.520,0.000,43.337,36,0,11,1,0,0,555.000,0.078414,0.000000,0.078085,0


In [17]:
df_sponsor_times_video_id_level.to_feather(
    DATA_PATH / "exploring_dataset_1_artifacts" / "sponsorTimes_filtered_videoID_level.feather"
)